In [78]:
import numpy as np

def solve_cgains(mat, ref=0, fix_off=True):
    if mat.shape[0] == 1:
        # If we only have one antenna, then this whole thing can be bypassed.
        return np.ones([1], dtype=mat.dtype)

    eig_vals, eig_vecs = np.linalg.eig(mat)
    soln_idx = eig_vals.real.argmax()
    gain_soln = (eig_vecs[:, soln_idx] * np.sqrt(eig_vals[soln_idx].real)).squeeze()

    if np.isfinite(gain_soln[ref]):
        # Apply reference antenna phase to the solutions
        gain_soln *= np.exp(-1j*np.angle(gain_soln[ref]))

        # Normalize to account for the fact the autos have been removed -- more
        # specifically, the real components (since the imag should be zero).
        if fix_off:
            gain_soln *= np.sqrt(2 * mat.size / ((2 * mat.size) - gain_soln.size))

        # Avoid numerical precision issues, explicitly zero out phaes for refant
        gain_soln.imag[ref] = 0

    return gain_soln

In [98]:
n_trials = 100
score_a = np.zeros(n_trials)
score_b = np.zeros(n_trials)
score_c = np.zeros(n_trials)
score_d = np.zeros(n_trials)
scale_fac = 0.2
gain_vec = np.ones(8, dtype='c16') * scale_fac
#gain_vec[-1] = 0.5
corr_matrix1 = np.ones((8, 8), dtype='c16')
corr_matrix2 = np.zeros((8, 8), dtype='c16')
for trial in range(n_trials):
    for idx in range(8):
        for jdx in range(8):
            if jdx < idx:
                corr_matrix1[idx, jdx] = np.conj(corr_matrix1[jdx, idx])
                corr_matrix2[idx, jdx] = np.conj(corr_matrix2[jdx, idx])
            elif jdx != idx:
                spec = (gain_vec[idx] * gain_vec[jdx]) + np.random.randn(32768).view('c16')
                corr_matrix1[idx, jdx] = np.mean(spec/abs(spec))
                corr_matrix2[idx, jdx] = np.median(spec.real) + np.median(spec.imag)*1j
    soln1 = solve_cgains(corr_matrix1, fix_off=False)
    soln2 = solve_cgains(corr_matrix2, fix_off=True)
    score_a[trial] = np.mean(np.exp(1j*np.angle(soln1))*gain_vec).real**2.0
    score_b[trial] = np.mean(np.exp(1j*np.angle(soln2))*gain_vec).real**2.0
    score_c[trial] = abs(np.sum(soln1))/np.sum(abs(soln1))
    score_d[trial] = abs(np.sum(soln2))/np.sum(abs(soln2))

print(np.mean(score_a)/(scale_fac**2), np.mean(score_b)/(scale_fac**2), np.mean(score_c), np.mean(score_d))

0.9895310904700279 0.9879683725964372 0.9974276371035206 0.9969355590672664


In [99]:
print(abs(soln1/soln2))
print(abs(soln2/gain_vec))

[1.88063355 2.01884891 2.11246521 1.91154418 1.88408921 1.81125978
 1.96490053 2.01590965]
[0.98807236 0.92707591 0.86392045 1.01322933 1.14527806 1.04562653
 1.03195892 0.88633924]


In [9]:
data_arr = ones((768, 2, 32768), dtype='<f4')
cross_idx = arange(96, 768)
auto1_idx = arange(96, 768) % 96
auto2_idx = (arange(96, 768) // 32) % 96
auto_sb_idx = 1

In [18]:
%timeit normalize_data(data_arr.view('<c8'), cross_idx, auto1_idx, auto2_idx, auto_sb_idx)

15.6 ms ± 432 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
import numpy as np


dummy_data = np.ones((512 * 6, 16384 * 2), dtype='<f4')
dummy_data[:, ::2] = 0
dummy_phase = np.ones((512 * 6), dtype='<c8') * (0.6 + 0.8j)

def complex_nan_to_num(arr):
    arr[np.isnan(arr)] == 0
    return arr


In [22]:
%%timeit
new_dummy = np.zeros((16384, 8, 8), dtype='c16')
for idx in range(64):
    if (idx % 8) != (idx // 8):
        baseline_data = dummy_data[idx<<2]
        complex_data = baseline_data[0::2] + 1j * baseline_data[1::2]
        new_dummy[:, idx % 8, idx // 8] = complex_data

np.mean(np.divide(new_dummy, abs(new_dummy), where=(new_dummy != 0)), axis=0)


36 ms ± 673 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit
for idx in range(len(dummy_data)):
    # Extract complex correlator data
    baseline_data = dummy_data[idx]
    complex_data = baseline_data[0::2] + 1j * baseline_data[1::2]

    # Apply phases
    complex_data = complex_data * dummy_phase[idx]

    # Reformat to original format and store
    baseline_data = np.vstack((complex_data.real,complex_data.imag)).flatten(order='F')
    dummy_data[idx] = baseline_data

666 ms ± 5.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
for idx in range(len(dummy_data)):
    # Extract complex correlator data
    baseline_data = dummy_data[idx].view('<c8')
    baseline_data *= dummy_phase[idx]

85.2 ms ± 437 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
np.abs(dummy_data[0, 0:2].view('c8'))

array([1.0000023], dtype=float32)

In [158]:
import numpy as np

def gain_solve(vis_data, ant1arr, ant2arr, vis_model=None, vis_noise=None, ref_ant=None, tol_spec=1e-5):
    complex_dtype = "%sc%i" % (vis_data.dtype.byteorder, vis_data.dtype.itemsize)
    float_dtype = "%sf%i" % (vis_data.dtype.byteorder, vis_data.dtype.itemsize // 2)

    # This is some basic import stuff
    ref_ant = 0

    # Basic data screening, make sure that everything looks okay and that
    # we have the relevant metadata on array shapes.

    # Enforce the existence of a model
    if vis_model is None:
        vis_model = np.ones_like(vis_data)
    elif (True if (not isinstance(vis_model, np.ndarray)) else (vis_model.size == 1)):
        vis_model = np.ones_like(vis_data) * vis_model
    elif vis_model.shape != vis_data.shape:
        raise ValueError("vis_model must be the same shape as vis_data.")

    # Enforce the existence of a data weights
    if vis_noise is None:
        vis_noise = np.ones_like(vis_data)
    elif (True if (not isinstance(vis_noise, np.ndarray)) else (vis_noise.size == 1)):
        vis_noise = np.ones_like(vis_data) * vis_noise
    elif vis_noise.shape != vis_data.shape:
        raise ValueError("vis_noise must be the same shape as vis_data.")


    data_mask = np.isfinite(vis_model) & np.isfinite(vis_noise) & np.isfinite(vis_data)
    if not np.all(data_mask):
        ant1arr = ant1arr[data_mask]
        ant2arr = ant2arr[data_mask]
        vis_data = vis_data[data_mask]
        vis_noise = vis_noise[data_mask]

    data_weight = (1. / vis_noise)

    # Prep data for LS processing, get some basic values
    n_ants = len(set(ant1arr).union(set(ant2arr)))
    n_base = vis_data.size

    # Build up the coordinate list for plugging things in
    common_coords = np.empty(4 * n_base, dtype=int)
    base_idx = np.arange(n_base)
    common_coords[0::4] = base_idx + (n_base * ((2 * ant1arr) - (ant1arr > ref_ant)))
    common_coords[2::4] = base_idx + (n_base * ((2 * ant2arr) - (ant2arr > ref_ant)))
    common_coords[1::2] = common_coords[::2] + n_base
    base_idx = np.repeat(base_idx, 4)

    pos_coords = np.empty(4 * n_base, dtype=int)
    pos_coords[0::4] = (2 * ant2arr)
    pos_coords[2::4] = (2 * ant1arr)
    pos_coords[1::2] = pos_coords[::2] + 1

    neg_coords = np.empty(4 * n_base, dtype=int)
    neg_coords[1::4] = (2 * ant2arr)
    neg_coords[3::4] = (2 * ant1arr)
    neg_coords[0::2] = neg_coords[1::2] + 1

    #sign_vals = np.tile([-1, 1, 1, -1], dtype=float_dtype), n_base)
    sign_vals = np.ones(4 * n_base, dtype=float_dtype)
    sign_vals[0::4] = -1.
    sign_vals[3::4] = -1.

    common_mask = np.ones(4 * n_base, dtype=bool)
    common_mask[1::4] = ant1arr != ref_ant
    common_mask[3::4] = ant2arr != ref_ant

    common_coords = common_coords[common_mask]
    pos_coords = pos_coords[common_mask]
    neg_coords = neg_coords[common_mask]
    sign_vals = sign_vals[common_mask]
    base_idx = base_idx[common_mask]

    # toTest is the change in solution from value to value.
    tol_test = 1
    cycle=0

    # cycleFinLim is the number of iterations to try before bailing. nParams^2
    # appeared to give the best dynamic results (i.e. if a solution is possible,
    # Monte Carlo tests showed that it always appeared in this number of cycles).
    cycle_lim = 64 if (n_ants < 5) else (((2 * n_ants) - 1) ** 2)

    # gain_guess provides a first guess based on the baselines of the refAnt. This
    # step seems to cut convergence time in half. Needs to be split into separate
    # real and imag components for the solver, though.
    # TODO: Put guess code here
    ant_guess = np.ones(n_ants, dtype=complex_dtype)

    guess_mask = np.ones(n_ants * 2, dtype=bool)
    guess_mask[(2 * ref_ant) + 1] = False

    del_guess = np.zeros(n_ants, dtype=complex_dtype)
    tot_mask = np.zeros(((2 * n_ants) - 1) * n_base, dtype=complex_dtype)

    # Here is the solver loop.
    # Complete either by hitting cycle limit or hitting tolerance target
    last_val = 1e300
    tol_test = 1
    damp_fac = 1
    while (cycle < cycle_lim):

        # Create some gain correction factors, which will be used later
        # to determine the data residuals
        gain_app = ant_guess[ant1arr] * np.conj(ant_guess[ant2arr])
        cycle += 1

        # Create a "generic" mask based of the first-order Taylor expansion
        # of the gains.    
        tot_mask.real[common_coords] = ant_guess.view(float_dtype)[pos_coords]
        tot_mask.imag[common_coords] = ant_guess.view(float_dtype)[neg_coords] * sign_vals
        tot_mask[common_coords] *= (data_weight * vis_model)[base_idx]

        # Calculate the true x-values for each table entry, corrected for noise
        x_matrix = tot_mask.reshape((2 * n_ants) - 1, n_base).view(float_dtype)

        # Calculate the true residuals, corrected for noise, convert to floats
        # so that we can evaluate the matrix more easily
        y_vec = ((vis_data - (vis_model * gain_app)) * data_weight).view(float_dtype)
        curr_val = np.sum(y_vec**2)

        if (np.abs(last_val - curr_val) < (curr_val * tol_spec)) and (
            damp_fac > (1. - (1 / (2 * n_ants - 1)))
        ):
            break
        last_val = curr_val

        alpha_matrix = x_matrix @ x_matrix.T
        beta_matrix = x_matrix @ y_vec
        del_guess.view(float_dtype)[guess_mask] = np.linalg.solve(alpha_matrix, beta_matrix)

        # Add new deltas to solutions, but dampen it in the case of large changes to
        # prevent positive feedback loops. The formula below is based on how large
        # the deltas need to be in order for second-order effects to create such
        # loops. Increases the number of cycles requires by convergence by up to 50%,
        # but greatly enhances stability.
        damp_fac = (1. + ((np.abs(del_guess) / np.abs(ant_guess)).sum()) / n_ants) ** -1
        ant_guess = ant_guess + (del_guess * damp_fac)

        # Prevent a degenerate case where the phase of the refant becomes 180
        # instead of zero (i.e. a negative, real number for the gains soln).
        if  ant_guess[ref_ant].real < 0:
            ant_guess *= -1

    # Finally, handle the results

    # Are there any NaNs, Infs or zeros in the gains soln?
    err_check = not (np.all(ant_guess) and np.all(np.isfinite(ant_guess)))

    # If we reached the cycle lim or there was an error in the check above,
    # mark solutions as bad.
    if (cycle == cycle_lim) or err_check:
        gain_table = np.zeros(n_ants * 2)
        gain_covar = np.zeros((n_ants * 2, n_ants * 2))
        gain_errs = np.zeros(n_ants * 2)
    else:
        gain_table = ant_guess
        covar_matrix = np.linalg.inv(alpha_matrix)
        rchi2_val =  np.dot(y_vec, y_vec) / ((2 * len(vis_data)) - ((2 * n_ants) - 1))
        gain_var = np.zeros(2 * n_ants, dtype=float_dtype)
        gain_var[guess_mask] = np.diag(covar_matrix) * rchi2_val
        gain_var[2 * ref_ant] *= 2
        gain_errs = np.sqrt(gain_var.reshape(-1, 2).sum(-1) * 0.5)

    return gain_table, gain_errs, cycle

In [159]:
def test_loop_gains():
    bad_count = 0
    real_bad_count = 0
    total_count = 16384
    cycle_count = 0
    ant1arr = np.array([idx for idx in range(8) for jdx in range(idx + 1, 8)])
    ant2arr = np.array([jdx for idx in range(8) for jdx in range(idx + 1, 8)])
    vis_model = np.ones(28)
    vis_noise = np.ones(28)
    err_check = 0.0
    for idx in range(total_count):
        vis_data = 0.1 * (0.5 ** 2.0) * (np.random.randn(28) + (1j * np.random.randn(28))) + 1.0
        gain_table, gain_errs, cycle = gain_solve(vis_data, ant1arr, ant2arr, vis_model=vis_model, vis_noise=vis_noise)
        cycle_count += cycle
        #gain_grade = np.mean(np.abs(gain_table - 1)**2)**0.5
        #if gain_grade > 1e3:
        #    real_bad_count += 1
        #if gain_grade > 1:
        #    bad_count += 1
        #err_check += np.sum(np.abs((gain_table - 1.) / gain_errs)**2.) / 16.
        
    #print(cycle_count / total_count, 100*bad_count / total_count, 100*real_bad_count / total_count)
    #print(err_check / total_count)

In [160]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [161]:
%lprun -f gain_solve test_loop_gains()

Timer unit: 1e-06 s

Total time: 10.4707 s
File: /var/folders/6k/9m18n_s947n5ts95gb8v5v180000gn/T/ipykernel_24591/3397059766.py
Function: gain_solve at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def gain_solve(vis_data, ant1arr, ant2arr, vis_model=None, vis_noise=None, ref_ant=None, tol_spec=1e-5):
     4     16384      48894.0      3.0      0.5      complex_dtype = "%sc%i" % (vis_data.dtype.byteorder, vis_data.dtype.itemsize)
     5     16384      28610.0      1.7      0.3      float_dtype = "%sf%i" % (vis_data.dtype.byteorder, vis_data.dtype.itemsize // 2)
     6                                           
     7                                               # This is some basic import stuff
     8     16384      15595.0      1.0      0.1      ref_ant = 0
     9                                           
    10                                               # Basic data screening, make sure that everything loo